In [71]:
import cv2
import numpy as np
from IPython.display import clear_output

In [223]:
def rlencode(x, dropna=False):
    """
    Run length encoding.
    Based on http://stackoverflow.com/a/32681075, which is based on the rle 
    function from R.
    
    Parameters
    ----------
    x : 1D array_like
        Input array to encode
    dropna: bool, optional
        Drop all runs of NaNs.
    
    Returns
    -------
    start positions, run lengths, run values
    
    """
    where = np.flatnonzero
    x = np.asarray(x)
    n = len(x)
    if n == 0:
        return (np.array([], dtype=int), 
                np.array([], dtype=int), 
                np.array([], dtype=x.dtype))

    starts = np.r_[0, where(~np.isclose(x[1:], x[:-1], equal_nan=True)) + 1]
    lengths = np.diff(np.r_[starts, n])
    values = x[starts]
    
    if dropna:
        mask = ~np.isnan(values)
        starts, lengths, values = starts[mask], lengths[mask], values[mask]
    
    return np.array((starts, lengths, values))

In [219]:
import sys
from collections import defaultdict
from IPython.display import display, clear_output

out_resolution = (128, 128)
(out_cols, out_rows) = out_resolution
np.set_printoptions(edgeitems=out_cols*out_rows)
np.set_printoptions(linewidth=out_cols+out_rows+2)

file_name = 'bad_apple.mp4'
cap = cv2.VideoCapture(file_name)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter(f'./out_{file_name}.avi', fourcc, 30, out_resolution, False)

nbytes = defaultdict(int)
iframe = 0

time_rle_buffer = np.ndarray(shape=out_resolution, dtype=bool)
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret is True:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        res_diff = int((width-height)/2)
        frame = frame[0:height, res_diff:width-res_diff]
        frame = cv2.resize(frame, out_resolution, fx=0, fy=0, interpolation=cv2.INTER_NEAREST)
        th, frame = cv2.threshold(frame, 0, 255, cv2.THRESH_OTSU)
        out.write(frame)
        
        nbytes['raw'] += frame.nbytes
        
        bool_frame = np.array(frame, dtype=bool)
        OD_frame = bool_frame.flatten()
        nbytes['1D frames'] += OD_frame.nbytes
        
        rle_starts, rle_lengths, rle_values = rlencode(OD_frame)
        nbytes['RLE 1D frames'] += rle_starts.nbytes +rle_lengths.nbytes + rle_values.nbytes
        
        time_rle_buffer = np.dstack((time_rle_buffer, bool_frame))
        
#         if iframe == 256:
#             clear_output(wait=True)
#             print(np.array2string(np.digitize(frame.flatten(), np.array([1])), separator=' ', prefix='', suffix=''))
        iframe += 1
    else:
        out.release()
        cap.release()

print("done!")
print(f"raw       : {nbytes['raw'] / 1000:,} kB")
print(f"flat      : {nbytes['1D frames'] / 1000:,} kB")
print(f"rle frame : {nbytes['RLE 1D frames'] / 1000:,} kB")

done! raw       : 107,675.648 kB
done! flat      : 107,675.648 kB
done! rle frame : 38,259.988 kB


In [224]:
a = np.copy(time_rle_buffer)
x,y,z = a.shape[0], a.shape[1], a.shape[2]
print(x, y, z)

a = a.reshape(x*y,z)
a = np.apply_along_axis(rlencode, 1, a)
a = a.reshape(x,y,z)

128 128 6573


ValueError: could not broadcast input array from shape (3,67) into shape (3,65)